In [ ]:
def log_progress(sequence, every=10):
    from ipywidgets import IntProgress
    from IPython.display import display

    progress = IntProgress(min=0, max=len(sequence), value=0)
    display(progress)
    
    for index, record in enumerate(sequence):
        if index % every == 0:
            progress.value = index
        yield record

In [ ]:
import requests    
def delete_tags(): 
    url = "http://localhost:8003/api/v1/tags/"
    r = requests.get(url)
    for t in log_progress(r.json()):
        idd = t['id']
        urld = f"http://localhost:8003/api/v1/tags/{idd}/"
        r = requests.delete(urld)     
delete_tags()

In [ ]:
import pandas as pd
dataset_path = '../data/path/dataset.csv'
df = pd.read_csv(dataset_path, on_bad_lines='skip')
df.head()

In [ ]:
import spacy
nlp = spacy.load("en_core_web_lg")

# query_text = "happiness love life"
query_text = "mood"
min_similarity = 0.35


sim = []
for idx in df.index:
    step = df.loc[idx, 'id']
    text = df.loc[idx, 'text']
    doc = nlp(text)
    query = nlp(query_text)
    for token in doc:
        if token.has_vector and query.has_vector:
            similarity = query.similarity(token)
            if similarity > min_similarity:
                # print(f"step: {step} added token.text {token.text}: {similarity:.2f}")
                sim.append(token.text.lower())
    if step > 2000:
        break
    
print(f"words: len(sim)")
# print(sim)

In [ ]:
import json
import requests
words = sim

for word in log_progress(sim, every=1):                
    url = "http://localhost:8003/api/v1/tags/"
    headers = {'content-type': 'application/json'}
    data = {'name': word, "count": 0}
    response = requests.post(url, data=json.dumps(data), headers=headers)
    # print(f"word: {word} added")

words = list(set(sim))
print(words[0:32])

In [ ]:
print(len(sim))
words = list(set(sim))

import spacy
import numpy as np
from sklearn.decomposition import PCA

# Calculate embeddings for each word
embeddings = np.array([nlp(word).vector for word in words])

# Use PCA to reduce the dimensions to 3
pca = PCA(n_components=3)
reduced_embeddings = pca.fit_transform(embeddings)
  
with open('../data/words.txt', 'w') as f:
    for word in words:
        f.write(f"{word}\n")
 
means = ""
with open('../data/means.txt', 'w') as f:
    for word in words:
        f.write(f"{word} ")
        means = means + f"{word} "
               
# Write the reduced embeddings to a .pts file
with open('../data/words.pts', 'w') as f:
    for embedding in reduced_embeddings:
        f.write(f"{embedding[0]} {embedding[1]} {embedding[2]}\n")   


In [ ]:
import spacy
nlp_lg = spacy.load("en_core_web_lg")

def get_hrases(text):
    hrases = []
    # Analyze noun hrases
    doc = nlp_lg(means)
    noun_hrases =  [chunk.text for chunk in doc.noun_chunks]
    for i in noun_hrases[0:64]:
        hrases.append(i)
    return hrases

hrases = get_hrases(means)
print(", ".join(hrases))